In [1]:
import pymc as pm
import numpy as np
import arviz as az
import pandas as pd
from aesara.tensor.subtensor import set_subtensor
import aesara.tensor as at

%load_ext lab_black
%load_ext watermark

# Prediction of time series

This example demonstrates ...

Adapted from [unit 10: sunspots.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/sunspots.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/sunspots.txt).

Associated lecture video: Unit 10 Lesson 7

## Problem statement

Sunspot numbers observed each year from 1770 to 1869. 

BUGS Book Page 258.

In [3]:
y = np.loadtxt("../data/sunspots.txt")

In [4]:
y

array([100.8,  81.6,  66.5,  34.8,  30.6,   7. ,  19.8,  92.5, 154.4,
       125.9,  84.8,  68.1,  38.5,  22.8,  10.2,  24.1,  82.9, 132. ,
       130.9, 118.1,  89.9,  66.6,  60. ,  46.9,  41. ,  21.3,  16. ,
         6.4,   4.1,   6.8,  14.5,  34. ,  45. ,  43.1,  47.5,  42.2,
        28.1,  10.1,   8.1,   2.5,   0. ,   1.4,   5. ,  12.2,  13.9,
        35.4,  45.8,  41.1,  30.4,  23.9,  15.7,   6.6,   4. ,   1.8,
         8.5,  16.6,  36.3,  49.7,  62.5,  67. ,  71. ,  47.8,  27.5,
         8.5,  13.2,  56.9, 121.5, 138.3, 103.2,  85.8,  63.2,  36.8,
        24.2,  10.7,  15. ,  40.1,  61.5,  98.5, 124.3,  95.9,  66.5,
        64.5,  54.2,  39. ,  20.6,   6.7,   4.3,  22.8,  54.8,  93.8,
        95.7,  77.2,  59.1,  44. ,  47. ,  30.5,  16.3,   7.3,  37.3,
        73.9])

In [6]:
t = np.array(range(100))
yr = t + 1770  # where is the professor using this?

## Model 1

In [35]:
with pm.Model() as m1:
    eps_0 = pm.Normal("eps_0", 0, tau=0.0001)

    theta = pm.Normal("theta", 0, tau=0.0001)
    c = pm.Normal("c", 0, tau=0.0001)
    sigma = pm.Uniform("sigma", 0, 100)
    tau = 1 / (sigma**2)

    _m = c + theta * np.roll(y, shift=-1)[:-1]
    m = set_subtensor(_m[0], y[0] - eps_0)

    _eps = y - m
    eps = set_subtensor(_eps[0], eps_0)

    pm.Normal("likelihood", mu=m, tau=tau, observed=y[:-1])

    trace = pm.sample(3000)

Auto-assigning NUTS sampler...
INFO:pymc:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO:pymc:Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc:Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps_0, theta, c, sigma]
INFO:pymc:NUTS: [eps_0, theta, c, sigma]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 12 seconds.
INFO:pymc:Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 12 seconds.


In [36]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
eps_0,0.077,21.726,-38.571,43.524,0.222,0.203,9621.0,7797.0,1.0
theta,0.817,0.060,0.709,0.932,0.001,0.000,7825.0,7267.0,1.0
c,8.506,3.528,1.995,15.239,0.040,0.028,7951.0,7430.0,1.0
sigma,21.966,1.626,19.051,25.030,0.017,0.012,9027.0,7453.0,1.0


## Model 1 using built in AR

docs are a little lacking here. will write up better instructions.

further reading:



- https://github.com/pymc-devs/pymc4/issues/198
- https://www.pymc-labs.io/blog-posts/bayesian-vector-autoregression/
- https://www.pymc.io/projects/examples/en/latest/time_series/AR.html
- https://discourse.pymc.io/t/vectorized-autoregressive-model/1449

In [38]:
with pm.Model() as m1_ar:
    eps_0 = pm.Normal("eps_0", 0, tau=0.0001)

    theta = pm.Normal("theta", 0, tau=0.0001)
    c = pm.Normal("c", 0, tau=0.0001)
    sigma = pm.Uniform("sigma", 0, 100)
    
    dist = pm.Normal.dist(mu=mu, sigma=sigma)
    pm.AR("likelihood", [c, theta], sigma=, observed=y)
    
    trace = pm.sample(3000)

ValueError: setting an array element with a sequence.